# Hotel Support Agent Tutorial

This notebook demonstrates the Agent Catalog hotel support agent based on the main.py implementation.

## Setup and Imports

Import all necessary modules from main.py.

In [1]:
import json
import os
import time
from datetime import timedelta

import agentc
import agentc_langchain
import dotenv
from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.management.buckets import CreateBucketSettings
from couchbase.management.search import SearchIndex
from couchbase.options import ClusterOptions
from langchain.agents import AgentExecutor, create_react_agent
from langchain.hub import pull
from langchain_couchbase.vectorstores import CouchbaseVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Import hotel data from the data module
from data.hotel_data import get_hotel_texts

# Load environment variables
dotenv.load_dotenv(override=True)

print("✅ Setup complete")

✅ Setup complete


## Environment Setup

Setup required environment variables with defaults.

In [2]:
def _set_if_undefined(var: str):
    if os.environ.get(var) is None:
        import getpass
        os.environ[var] = getpass.getpass(f"Please provide your {var}: ")

def setup_environment():
    required_vars = ['OPENAI_API_KEY', 'CB_HOST', 'CB_USERNAME', 'CB_PASSWORD', 'CB_BUCKET_NAME']
    for var in required_vars:
        _set_if_undefined(var)
    
    defaults = {
        'CB_HOST': 'couchbase://localhost',
        'CB_USERNAME': 'Administrator', 
        'CB_PASSWORD': 'password',
        'CB_BUCKET_NAME': 'vector-search-testing',
        'INDEX_NAME': 'vector_search_agentcatalog',
        'SCOPE_NAME': 'shared',
        'COLLECTION_NAME': 'agentcatalog'
    }
    
    for key, default_value in defaults.items():
        if not os.environ.get(key):
            os.environ[key] = input(f"Enter {key} (default: {default_value}): ") or default_value

setup_environment()
print("✅ Environment configured")

✅ Environment configured


## Agent Catalog and Span Setup

Initialize Agent Catalog and application span.

In [3]:
# Initialize Agent Catalog
catalog = agentc.Catalog()
application_span = catalog.Span(name="Hotel Search Agent")

print("✅ Agent Catalog initialized")

✅ Agent Catalog initialized


## Couchbase Connection

Setup Couchbase connection.

In [4]:
def setup_couchbase_connection():
    try:
        auth = PasswordAuthenticator(os.environ['CB_USERNAME'], os.environ['CB_PASSWORD'])
        options = ClusterOptions(auth)
        cluster = Cluster(os.environ['CB_HOST'], options)
        cluster.wait_until_ready(timedelta(seconds=10))
        print("Successfully connected to Couchbase")
        return cluster
    except Exception as e:
        raise ConnectionError(f"Failed to connect to Couchbase: {str(e)}")

with application_span.new("Couchbase Connection"):
    cluster = setup_couchbase_connection()

Successfully connected to Couchbase


## Collection Setup

Setup bucket, scope and collection.

In [5]:
def setup_collection(cluster, bucket_name, scope_name, collection_name):
    try:
        try:
            bucket = cluster.bucket(bucket_name)
            print(f"Bucket '{bucket_name}' exists")
        except Exception:
            print(f"Creating bucket '{bucket_name}'...")
            bucket_settings = CreateBucketSettings(
                name=bucket_name,
                bucket_type='couchbase',
                ram_quota_mb=1024,
                flush_enabled=True,
                num_replicas=0
            )
            cluster.buckets().create_bucket(bucket_settings)
            time.sleep(5)
            bucket = cluster.bucket(bucket_name)
            print(f"Bucket '{bucket_name}' created successfully")

        bucket_manager = bucket.collections()
        
        scopes = bucket_manager.get_all_scopes()
        scope_exists = any(scope.name == scope_name for scope in scopes)
        
        if not scope_exists and scope_name != "_default":
            print(f"Creating scope '{scope_name}'...")
            bucket_manager.create_scope(scope_name)
            print(f"Scope '{scope_name}' created successfully")

        collections = bucket_manager.get_all_scopes()
        collection_exists = any(
            scope.name == scope_name and collection_name in [col.name for col in scope.collections]
            for scope in collections
        )

        if not collection_exists:
            print(f"Creating collection '{collection_name}'...")
            bucket_manager.create_collection(scope_name, collection_name)
            print(f"Collection '{collection_name}' created successfully")

        collection = bucket.scope(scope_name).collection(collection_name)
        time.sleep(3)

        try:
            cluster.query(f"CREATE PRIMARY INDEX IF NOT EXISTS ON `{bucket_name}`.`{scope_name}`.`{collection_name}`").execute()
            print("Primary index created successfully")
        except Exception as e:
            print(f"Warning: Error creating primary index: {str(e)}")

        print("Collection setup complete. Using existing documents in the database.")
        
        return collection
    except Exception as e:
        raise RuntimeError(f"Error setting up collection: {str(e)}")

with application_span.new("Couchbase Collection Setup"):
    setup_collection(
        cluster, 
        os.environ['CB_BUCKET_NAME'], 
        os.environ['SCOPE_NAME'], 
        os.environ['COLLECTION_NAME']
    )

Bucket 'vector-search-testing' exists
Primary index created successfully
Collection setup complete. Using existing documents in the database.


## Vector Index Setup

Load and setup the vector search index definition.

In [6]:
def setup_vector_search_index(cluster, index_definition):
    try:
        scope_index_manager = cluster.bucket(os.environ['CB_BUCKET_NAME']).scope(os.environ['SCOPE_NAME']).search_indexes()
        
        existing_indexes = scope_index_manager.get_all_indexes()
        index_name = index_definition["name"]

        if index_name not in [index.name for index in existing_indexes]:
            print(f"Creating vector search index '{index_name}'...")
            search_index = SearchIndex.from_json(index_definition)
            scope_index_manager.upsert_index(search_index)
            print(f"Vector search index '{index_name}' created successfully")
        else:
            print(f"Vector search index '{index_name}' already exists")
    except Exception as e:
        raise RuntimeError(f"Error setting up vector search index: {str(e)}")

with application_span.new("Vector Index Setup"):
    try:
        with open('agentcatalog_index.json', 'r') as file:
            index_definition = json.load(file)
        print("Loaded vector search index definition from agentcatalog_index.json")
        setup_vector_search_index(cluster, index_definition)
    except Exception as e:
        raise ValueError(f"Error loading index definition: {str(e)}")

Loaded vector search index definition from agentcatalog_index.json
Vector search index 'vector_search_agentcatalog' already exists


## Vector Store Setup

Setup vector store and load hotel data.

In [7]:
def clear_collection_data(cluster):
    """Clear all documents from the collection to start fresh."""
    try:
        bucket_name = os.environ['CB_BUCKET_NAME']
        scope_name = os.environ['SCOPE_NAME']
        collection_name = os.environ['COLLECTION_NAME']
        
        # Delete all documents in the collection
        delete_query = f"DELETE FROM `{bucket_name}`.`{scope_name}`.`{collection_name}`"
        result = cluster.query(delete_query)
        
        print(f"Cleared existing data from collection {scope_name}.{collection_name}")
        
    except Exception as e:
        print(f"Warning: Could not clear collection data: {str(e)}. Continuing with existing data...")

def setup_vector_store(cluster):
    try:
        embeddings = OpenAIEmbeddings(
            api_key=os.environ['OPENAI_API_KEY'],
            model="text-embedding-3-small"
        )
        
        vector_store = CouchbaseVectorStore(
            cluster=cluster,
            bucket_name=os.environ['CB_BUCKET_NAME'],
            scope_name=os.environ['SCOPE_NAME'],
            collection_name=os.environ['COLLECTION_NAME'],
            embedding=embeddings,
            index_name=os.environ['INDEX_NAME'],
        )
        
        # Clear existing data before loading fresh hotel data
        clear_collection_data(cluster)
        
        # Use hotel data from the data module
        hotel_data = get_hotel_texts()
        
        try:
            vector_store.add_texts(texts=hotel_data, batch_size=10)
            print("Hotel data loaded into vector store successfully")
        except Exception as e:
            print(f"Warning: Error loading hotel data: {str(e)}. Vector store created but data not loaded.")
        
        return vector_store
    except Exception as e:
        raise ValueError(f"Error setting up vector store: {str(e)}")

with application_span.new("Vector Store Setup"):
    setup_vector_store(cluster)

Cleared existing data from collection shared.agentcatalog


## LLM Setup

Setup LLM with Agent Catalog callback.

In [8]:
with application_span.new("LLM Setup"):
    # Setup LLM with Agent Catalog callback
    llm = ChatOpenAI(
        api_key=os.environ['OPENAI_API_KEY'],
        model="gpt-4o",
        temperature=0,
        callbacks=[agentc_langchain.chat.Callback(span=application_span)]
    )
    print("✅ LLM setup complete")

✅ LLM setup complete


## Tool Loading

Load tools from Agent Catalog.

In [9]:
with application_span.new("Tool Loading"):
    # Load tools from Agent Catalog - they are now properly decorated
    tool_search = catalog.find("tool", name="search_vector_database")
    tool_details = catalog.find("tool", name="get_hotel_details")
    
    if not tool_search:
        raise ValueError("Could not find search_vector_database tool. Make sure it's indexed with 'agentc index tools/'")
    if not tool_details:
        raise ValueError("Could not find get_hotel_details tool. Make sure it's indexed with 'agentc index tools/'")
    
    from langchain_core.tools import Tool
    tools = [
        Tool(
            name=tool_search.meta.name,
            description=tool_search.meta.description,
            func=tool_search.func
        ),
        Tool(
            name=tool_details.meta.name, 
            description=tool_details.meta.description,
            func=tool_details.func
        )
    ]
    print(f"✅ Loaded {len(tools)} tools from Agent Catalog")

✅ Loaded 2 tools from Agent Catalog


## Agent Creation

Create the ReAct agent with prompt from Agent Catalog.

In [10]:
with application_span.new("Agent Creation"):
    # Get prompt from Agent Catalog
    hotel_prompt = catalog.find("prompt", name="hotel_search_assistant")
    if not hotel_prompt:
        raise ValueError("Could not find hotel_search_assistant prompt in catalog. Make sure it's indexed with 'agentc index prompts/'")
    
    # Create a custom prompt using the catalog prompt content
    from langchain_core.prompts import PromptTemplate
    
    # The prompt content is already properly structured with ReAct format
    prompt_content = hotel_prompt.content.strip()
    
    custom_prompt = PromptTemplate(
        template=prompt_content,
        input_variables=["input", "agent_scratchpad"],
        partial_variables={
            "tools": "\n".join([f"{tool.name}: {tool.description}" for tool in tools]),
            "tool_names": ", ".join([tool.name for tool in tools])
        }
    )
    
    agent = create_react_agent(llm, tools, custom_prompt)
    agent_executor = AgentExecutor(
        agent=agent, 
        tools=tools, 
        verbose=True, 
        handle_parsing_errors=True,
        max_iterations=5,
        return_intermediate_steps=True
    )
    print("✅ Hotel Search Agent is ready!")

✅ Hotel Search Agent is ready!


## Test Scenarios

Run the test scenarios from main.py.

In [11]:
# Test the agent with sample queries
print("\nHotel Search Agent is ready!")
print("Testing with sample queries...")

test_queries = [
    "Find me a luxury hotel with a pool and spa",
    "I need a beach resort in Miami", 
    "Get me details about Ocean Breeze Resort"
]

with application_span.new("Query Execution") as span:
    for query in test_queries:
        with span.new(f"Query: {query}") as query_span:
            print(f"\n🔍 Query: {query}")
            try:
                response = agent_executor.invoke({"input": query})
                query_span["response"] = response['output']
                print(f"✅ Response: {response['output']}")
                print("-" * 80)
            except Exception as e:
                query_span["error"] = str(e)
                print(f"❌ Error: {e}")
                print("-" * 80)

print("All tests completed!")


Hotel Search Agent is ready!
Testing with sample queries...

🔍 Query: Find me a luxury hotel with a pool and spa


> Entering new AgentExecutor chain...
Thought: I need to search for luxury hotels that have a pool and spa to meet the user's requirements.

Action: search_vector_database
Action Input: "luxury hotel with pool and spa"
HOTEL 1:
Match Score: 0.469
Hotel Details: Ocean Breeze Resort in Malibu, California. Luxury oceanfront resort with private beach, world-class spa, and championship golf course Price range: $400-$600 per night. Rating: 4.9/5. Amenities: Private Beach Access, Full-Service Spa, Championship Golf Course, Infinity Pool, Multiple Fine Dining Restaurants, Free WiFi

HOTEL 2:
Match Score: 0.440
Hotel Details: Grand Palace Hotel in Manhattan, New York City. Luxury 5-star hotel featuring elegant rooms with Manhattan skyline views Price range: $300-$500 per night. Rating: 4.8/5. Amenities: Rooftop Pool, World-class Spa, 24/7 Fitness Center, Michelin-starred Restauran